<a href="https://colab.research.google.com/github/dineshdyne/colab_notebooks/blob/main/TRAX_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trax
!pip install huggingface-hub more-itertools 

     |████████████████████████████████| 637 kB 5.0 MB/s 
     |████████████████████████████████| 4.9 MB 31.0 MB/s 
     |████████████████████████████████| 462 kB 44.1 MB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 


In [2]:
import pandas as pd
import tensorflow as tf
from trax.fastmath import numpy as np
from more_itertools import *


In [7]:

# Layers used to build the Deep Learning Models
import trax.layers as tl
# Utilities to build and download datasets
import trax.data as data
# Utilities for training
import trax.supervised as ts
# To import models and other utilities:
import trax


import os
import requests

In [8]:
trax.fastmath.set_backend('jax') #or tensorflow-numpy

# And this is how you set a TPU backend in Jax using Colab:
# Add TPU Address to Globals
if 'TPU_DRIVER_MODE' not in globals():
  url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
  resp = requests.post(url)
  TPU_DRIVER_MODE = 1
# Set TPU driver to be jax target.
from jax.config import config
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

In [9]:
os.environ['COLAB_TPU_ADDR']

'10.58.22.10:8470'

In [10]:
imdb = data.TFDS('imdb_reviews', keys=('text', 'label'), train=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2B3DC8/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2B3DC8/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/jax/_src/lib/xla_bridge.py:446: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "


In [11]:
# We can check the next value by using next(dataset). It returns the text in byte format and the sentiment (0 = neg, 1 = pos).
# Since it is a function, next(dataset_func()) will always return the first sentence in the dataset, unless we get the result from the function, which is the actual generator.
text_in_bytes, sent = next(imdb())
# To work with it, we have to decode the string, because it is in bytes format:
decoded = text_in_bytes.decode('utf-8')
# To avoid index errors in formatting:
max_len = 100 if len(decoded) > 100 else len(decoded)-1
print("The text, up to 100 characters, is:\n{}\nThe sentiment is: {}".format(decoded[:max_len], "Positive" if sent else "Negative"))

The text, up to 100 characters, is:
This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. 
The sentiment is: Negative
